In [1]:
# default_exp test

In [2]:
# export 
import numpy as np
import torch
import torchvision
import matplotlib.pyplot as plt
from time import time
from torchvision import datasets, transforms
from torch import nn, optim


In [3]:
# export
def get_test_resources(force_cpu=False):
    if force_cpu:
        device = torch.device("cpu")
    else:
        device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    transform = transforms.Compose([transforms.ToTensor(),
                                  transforms.Normalize((0.5,), (0.5,)),
                                  ])

    trainset = datasets.MNIST('resources/mnist/train', download=True, train=True, transform=transform)
    valset = datasets.MNIST('resources/mnist/test', download=True, train=False, transform=transform)
    trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)
    valloader = torch.utils.data.DataLoader(valset, batch_size=64, shuffle=True)

    dls = {
        'train':trainloader,
        'test':valloader,
        'valid':valloader,
    }

    input_size = 784
    hidden_sizes = [128, 64]
    output_size = 10
    
    class Loss(nn.Module):
        def __init__(self):
            super().__init__()
            self.loss_fn = nn.CrossEntropyLoss()
        def forward(self,x,y):
            loss = self.loss_fn(x,y)
            return loss,loss

    model = nn.Sequential(nn.Linear(input_size, hidden_sizes[0]),
                          nn.ReLU(),
                          nn.Linear(hidden_sizes[0], hidden_sizes[1]),
                          nn.ReLU(),
                          nn.Linear(hidden_sizes[1], output_size)
                         )
    model = model.to(device)
    optimizer = optim.SGD(model.parameters(), lr=0.003, momentum=0.9)
    loss_func = Loss()
    return model,dls,loss_func,optimizer,device


In [1]:
from nbdev.export import *
notebook2script()

Converted 00_core.ipynb.
Converted 02_learner.ipynb.
Converted 05_progressbar.ipynb.
Converted 99_tests.ipynb.
Converted index.ipynb.


---

In [5]:
%load_ext autoreload
%autoreload 2

In [6]:
from nabla2.learner import Learner
from nabla2.test import get_test_resources

In [7]:
model,dls,loss_func,optimizer,device = get_test_resources(True)

def foo(p,g):
    return -1.0


learner = Learner(model,dls,loss_func,optimizer,device=device,eval_sets={'valid':foo,'train':foo,'test':[foo],'twst1':foo})

def on_batch_begin(data_batch,phase='train'):
    xb,yb = data_batch
    xb = xb.view(-1, 784)
    return xb,yb
learner.on_batch_begin=on_batch_begin


>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
24/04/2020 12:01:08


In [8]:
learner.fit(2)

/home/deva/WORK/nabla2/nabla2/learner.py:208: RuntimeWarning: Mean of empty slice.
  average_valid_loss = _redux(valid_losses).mean(axis=0)
/home/deva/.virtualenv/image_comp/lib/python3.6/site-packages/numpy/core/_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


          error  foo                        loss                                                       
EPOCH     twst1 test train valid            test             train           valid
1      na in ds -1.0  -1.0  -1.0  [0.304, 0.304]  [[0.643, 0.643]]  [0.304, 0.304]
2      na in ds -1.0  -1.0  -1.0  [0.242, 0.242]  [[0.289, 0.289]]  [0.241, 0.241]                     


In [9]:
learner.eval_metrics()

defaultdict(dict,
            {'valid': {'foo': -1.0, 'loss': array([0.242, 0.242])},
             'train': {'foo': -1.0, 'loss': array([0.245, 0.245])},
             'test': {'foo': -1.0, 'loss': array([0.24, 0.24])},
             'twst1': {'error': 'na in ds'}})